In [1]:
FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader(FILE_PATH)

# 문서 로딩
docs = loader.load()

In [3]:
print(docs[10].page_content[:300])

SPRi AI Brief |
2023-12월호
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
KEY Contents
n 코히어와 12개 기관이 광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태,
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의
구성과 계보도 추적 가능
£데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어(Cohere)


# 멀티 로더 폴백 전략

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader, PDFMinerLoader, PyMuPDFLoader

def load_pdf_with_fallback(file_path: str):
    loaders = [
        ("PDFPlumber", PDFPlumberLoader),
        ("PDFMiner", PDFMinerLoader),
        ("PyMuPDF", PyMuPDFLoader)
    ]
    
    for name, LoaderClass in loaders:
        try:
            print(f"시도 중: {name}")
            loader = LoaderClass(file_path)
            docs = loader.load()
            
            # 기본 검증만
            if docs and len(docs[0].page_content.strip()) > 50:
                print(f"✓ {name} 성공 - {len(docs)}페이지 추출")
                return docs, name
        except Exception as e:
            print(f"✗ {name} 실패: {str(e)[:50]}")
    
    raise Exception("모든 로더 실패")

# 각 로더별 추출 품질 비교 테스트

In [5]:
def compare_loaders(file_path: str):
    loaders = {
        "PDFPlumber": PDFPlumberLoader,
        "PDFMiner": PDFMinerLoader,
        "PyMuPDF": PyMuPDFLoader
    }
    
    results = {}
    for name, LoaderClass in loaders.items():
        try:
            loader = LoaderClass(file_path)
            docs = loader.load()
            
            text = docs[0].page_content
            results[name] = {
                "페이지수": len(docs),
                "총글자수": len(text),
                "첫100자": text[:100]
            }
        except Exception as e:
            results[name] = {"에러": str(e)}
    
    return results

In [6]:
# 현재 PDF로 비교 테스트
file_path = "./data/SPRi_AI_Brief_2023년12월호_F.pdf"
results = compare_loaders(file_path)

for loader_name, info in results.items():
    print(f"\n{'='*50}")
    print(f"{loader_name}")
    print(f"{'='*50}")
    for key, value in info.items():
        print(f"{key}: {value}")


PDFPlumber
페이지수: 23
총글자수: 12
첫100자: 12
2023년 월호


PDFMiner
페이지수: 1
총글자수: 31547
첫100자: 2023년  12월호
2023년  12월호

Ⅰ.  인공지능  산업  동향  브리프

  1.  정책/법제 

      ▹  미국,  안전하고  신뢰할  수  있는  AI  개

PyMuPDF
페이지수: 23
총글자수: 10
첫100자: 2023년 12월호


In [7]:
from langchain_community.document_loaders import PDFMinerLoader, PDFPlumberLoader, PyMuPDFLoader

def load_pdf_optimized(file_path: str):
    """
    AutoRAG 실험 결과를 반영한 우선순위:
    1순위: PDFMiner (이 문서에서 최고 성능)
    2순위: PDFPlumber (일반적으로 우수)
    3순위: PyMuPDF (폴백용)
    """
    loaders = [
        ("PDFMiner", PDFMinerLoader),
        ("PDFPlumber", PDFPlumberLoader),
        ("PyMuPDF", PyMuPDFLoader)
    ]
    
    for name, LoaderClass in loaders:
        try:
            loader = LoaderClass(file_path)
            docs = loader.load()
            
            # 실제 텍스트가 추출되었는지 확인
            total_text = "".join([doc.page_content for doc in docs])
            char_count = len(total_text.strip())
            
            if char_count > 100:  # 최소 100자 이상
                print(f"✓ {name} 성공: {len(docs)}개 문서, {char_count:,}자 추출")
                return docs, name
            else:
                print(f"✗ {name} 텍스트 부족: {char_count}자만 추출")
                
        except Exception as e:
            print(f"✗ {name} 에러: {str(e)[:50]}")
    
    raise Exception("모든 로더가 실패했거나 텍스트 추출 실패")

In [8]:
# 최적화된 로더 사용
docs, used_loader = load_pdf_optimized(FILE_PATH)

print(f"\n사용된 로더: {used_loader}")
print(f"문서 개수: {len(docs)}")
print(f"\n첫 500자 미리보기:")
print(docs[0].page_content[:1300])

✓ PDFMiner 성공: 1개 문서, 31,547자 추출

사용된 로더: PDFMiner
문서 개수: 1

첫 500자 미리보기:
2023년  12월호
2023년  12월호

Ⅰ.  인공지능  산업  동향  브리프

  1.  정책/법제 

      ▹  미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표    ························· 1

      ▹  G7,  히로시마  AI  프로세스를  통해  AI  기업  대상  국제  행동강령에  합의 ··························· 2

      ▹  영국  AI  안전성  정상회의에  참가한  28개국,  AI  위험에  공동  대응  선언 ··························· 3

      ▹  미국  법원,  예술가들이  생성  AI  기업에  제기한  저작권  소송  기각 ····································· 4

      ▹  미국  연방거래위원회,  저작권청에  소비자  보호와  경쟁  측면의  AI  의견서  제출 ················· 5

      ▹  EU  AI  법  3자  협상,  기반모델  규제  관련  견해차로  난항 ··················································· 6

  2.  기업/산업 

      ▹  미국  프런티어  모델  포럼,  1,000만  달러  규모의  AI  안전  기금  조성 ································ 7

      ▹  코히어,  데이터  투명성  확보를  위한  데이터  출처  탐색기  공개    ······································· 8

      ▹  알리바바  클라우드,  최신  LLM  ‘통이치엔원  2.0’  공개  ···············································